In [1]:
# -*- coding:utf-8 -*-
import gc
import os
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame

In [2]:
def judge_time(time_on,time_down):
  if (time_on+5) > time_down:
    return True
  else:
    return False

In [3]:
# 运动学片段提取
def problem1(df):
    time_list = list(df['时间'].values)
    speed_list = list(df['GPS车速'].values)
    i_start = 0
    i_end = 0
    loc_list = []
    temp_an = 0

    while temp_an < (len(time_list)-1):
      while speed_list[temp_an] == 0 and temp_an < (len(time_list)-1):
        temp_an+=1
      if speed_list[temp_an-1] == 0:
        i_start = temp_an
        time_on = time_list[temp_an-1]
        time_down = time_list[temp_an]
        while speed_list[temp_an]!=0 and judge_time(time_on,time_down)==True and temp_an < (len(time_list)-1):
          time_on = time_list[temp_an]
          time_down = time_list[temp_an+1]
          temp_an+=1
        if speed_list[temp_an-1]!=0 and speed_list[temp_an]==0:
          i_end = temp_an
          loc_list.append([i_start,i_end])
      temp_an+=1

    total = 0
    for loc in loc_list:
        total+=(loc[1]-loc[0])

#     print(total)
    return loc_list

In [4]:
# 片段时长
def duration_computing(data, loc_list):
    v = data['GPS车速']
    t = data['时间']
    duration = []
    for [start, end] in loc_list:
        duration.append(t.iloc[end] - t.iloc[start])
    return duration

In [5]:
# 怠速时长
def idleSpeed_duration_computing(data, loc_list):
    v = data['GPS车速']
    t = data['时间']
    idleSpeed_duration = []
    for [start, end] in loc_list:
        idleSpeed_end = start
        while v.iloc[idleSpeed_end] == 0:
            idleSpeed_end += 1
        idleSpeed_duration.append(t.iloc[idleSpeed_end] - t.iloc[start])
    return idleSpeed_duration

In [6]:
# 运行时间
def run_duration_computing(duration, idleSpeed_duration):
    run_duration = [duration[i] - idleSpeed_duration[i] for i in range(len(duration))]
    return run_duration

In [7]:
def max_v(data, loc_list):
    v = data['GPS车速']
    v_max = []
    for [start, end] in loc_list:
        v_max.append(max(v.iloc[start:end]))
    return v_max

In [8]:
from scipy import integrate
def distance_computing(data, loc_list):
    v = data['GPS车速']
    t = data['时间']
    distance = []
    for [start, end] in loc_list:
        distance.append(integrate.trapz(v.iloc[start:end], t.iloc[start:end]))
    return distance

In [9]:
import matplotlib.pyplot as plt
def draw(file, loc_list):
    for [start, end] in loc_list:
        data = file[start:end]
        plt.rcParams['savefig.dpi'] = 300 #图片像素
        plt.rcParams['figure.dpi'] = 300 #分辨率
        data.plot(x = '时间', y = 'GPS车速')
        path = 'file1_v/' + str(start) + '-' + str(end) + '.png'
        plt.savefig(path)

In [10]:
def mean_v(distance, duration):
    v_mean = [distance[i] / duration[i] for i in range(len(distance))]
    return v_mean

In [11]:
def acceleration_computing(data, loc_list):
    v = data['GPS车速']
    t = data['时间']
    acc_plus_time = []
    acc_reduce_time = []
    acc_max = []
    acc_min = []
    acc_mean_plus = []
    acc_mean_reduce = []
    acc_std_plus = []
    acc_std_reduce = []
    acc_std = []
    for [start, end] in loc_list:
        a = []
        a_plus_time = 0
        a_reduce_time = 0
        for i in range(start, end):
            if v.iloc[i+1] - v.iloc[i] > 0:
                a_plus_time += t.iloc[i+1] - t.iloc[i]
            if v.iloc[i+1] - v.iloc[i] < 0:
                a_reduce_time += t.iloc[i+1] - t.iloc[i]
            a.append((v.iloc[i+1] - v.iloc[i]) / (t.iloc[i+1] - t.iloc[i]))
        # 最大加速度
        a_max = max(a)
        # 最大减速度
        a_min = min(a)
        # 加速
        a_plus = [i for i in a if i > 0]
        # 减速
        a_reduce = [i for i in a if i < 0]
        # 平均加速
        a_mean_plus = np.mean(a_plus)
        # 平均减速
        a_mean_reduce = np.mean(a_reduce)
        # 加速标准差
        a_std_plus = np.std(a_plus, ddof = 1)
        # 减速标准差
        a_std_reduce = np.std(a_reduce, ddof = 1)
        a_std = np.std(a, ddof = 1)
        acc_plus_time.append(a_plus_time)
        acc_reduce_time.append(a_reduce_time)
        acc_max.append(a_max)
        acc_min.append(a_min)
        acc_mean_plus.append(a_mean_plus)
        acc_mean_reduce.append(a_mean_reduce)
        acc_std_plus.append(a_std_plus)
        acc_std_reduce.append(a_std_reduce)
        acc_std.append(a_std)
    return acc_plus_time, acc_reduce_time, acc_max,acc_min,acc_mean_plus,acc_mean_reduce,acc_std_plus,acc_std_reduce,acc_std

In [12]:
def get_feature(data, loc_list):
    duration = duration_computing(data, loc_list)
    idleSpeed_duration = idleSpeed_duration_computing(data, loc_list)
    distance = distance_computing(data, loc_list)
    run_duration = run_duration_computing(duration, idleSpeed_duration)
    v_max = max_v(data, loc_list)
    v_mean = mean_v(distance, duration)
    v_mean_nonIdle = mean_v(distance, run_duration)
    acc_plus_time, acc_reduce_time, acc_max,acc_min,acc_mean_plus,acc_mean_reduce,acc_std_plus,acc_std_reduce,acc_std = acceleration_computing(data, loc_list)
    feature = { 'duration' : duration,
               'distance': distance,
               'idleSpeed_duration': idleSpeed_duration,
               'run_duration': run_duration,
               'v_max': v_max,
               'v_mean': v_mean,
               'v_mean_nonIdle': v_mean_nonIdle,
               'acc_plus_time': acc_plus_time, 
               'acc_reduce_time': acc_reduce_time, 
               'acc_max': acc_max,
               'acc_min':acc_min,
               'acc_mean_plus': acc_mean_plus,
               'acc_mean_reduce':acc_mean_reduce,
               'acc_std_plus': acc_std_plus,
               'acc_std_reduce': acc_std_reduce,
               'acc_std': acc_std
              }
    df_feature = DataFrame(feature)
    return df_feature

In [13]:
file1 = pd.read_csv('process_carlosfile1.csv')
loc_list = problem1(file1)

In [14]:
feature = get_feature(file1, loc_list)

/home/liumeng/.conda/envs/kaggle/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/liumeng/.conda/envs/kaggle/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/liumeng/.conda/envs/kaggle/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/liumeng/.conda/envs/kaggle/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/liumeng/.conda/envs/kaggle/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)


In [16]:
feature.describe()

,duration,distance,idleSpeed_duration,run_duration,v_max,v_mean,v_mean_nonIdle,acc_plus_time,acc_reduce_time,acc_max,acc_min,acc_mean_plus,acc_mean_reduce,acc_std_plus,acc_std_reduce,acc_std
count,405.000000,405.000000,405.0,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,294.000000,405.000000,231.000000,49.000000,315.000000
mean,19.335802,185.312469,0.0,19.335802,13.245185,4.266509,4.266509,7.227160,11.595062,2.310475,-1.139709,1.613670,-1.058212,1.161131,0.869561,1.519232
std,24.411607,401.368480,0.0,24.411607,15.040879,6.466772,6.466772,10.621422,17.286719,2.865479,0.835445,1.437344,0.778797,1.062896,0.659952,1.311124
min,3.000000,0.000000,0.0,3.000000,0.100000,0.000000,0.000000,0.000000,3.000000,-2.100000,-4.258333,0.033333,-4.258333,0.000000,0.000000,0.014434
25%,6.000000,0.350000,0.0,6.000000,2.700000,0.057143,0.057143,0.000000,5.000000,0.000000,-1.600000,0.781250,-1.484211,0.522010,0.312305,0.736457
50%,10.000000,12.300000,0.0,10.000000,6.500000,1.257143,1.257143,2.000000,7.000000,1.700000,-0.960000,1.324296,-0.883333,0.903011,0.791960,1.227291
75%,22.000000,117.150000,0.0,22.000000,17.500000,5.515217,5.515217,9.000000,12.000000,4.000000,-0.500000,2.000000,-0.500000,1.389130,1.218917,1.889887
max,234.000000,2660.750000,0.0,234.000000,64.300000,32.848765,32.848765,55.000000,228.000000,12.300000,-0.020000,11.500000,-0.020000,6.157922,2.607877,9.192388
